# Test Benchmark

## Buffer

buffer store function (multi done scenario)

In [ ]:
from replay_buffer import ReplayBuffer
from agent import AgentSAC
from attrdict import AttrDict
from envs import ReachToyEnv
import torch
args = AttrDict(learning_rate=0.01, env_num=2, eval_gap=0, eval_steps_per_env=10, cwd='')
agent = AgentSAC(10,4,2,max_env_step=5, goal_dim=2,gpu_id=-1, info_dim=4+4,args=args)
env = ReachToyEnv(max_step=5, gpu_id=-1)
buffer = ReplayBuffer(gpu_id=-1,
                      max_len=10,
                      state_dim=4,
                      action_dim=2,
                      goal_dim=2,
                      reward_fn = env.compute_reward,
                      info_dim=4+4) # add traj index

In [ ]:
traj = agent.explore_env(env, target_steps_per_env=12, random_mode=True, buffer = buffer)

In [ ]:
buffer.buf_state, buffer.buf_other[...,-1], buffer.buf_other[...,4+2:4+4]

In [ ]:
buffer.update_now_len()
rew, mask, act, s, ns, i= buffer.sample_batch(10, her_rate=1, indices=torch.arange(start=5,end=9))

In [ ]:
s, rew

HER buffer

In [ ]:
buffer.buf_other[:20][..., :2]